In [13]:
import os
import os.path as path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


In [14]:
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataExample.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('seção,subtitulo,post\n')  # Cabeçalho compatível com os dados
    return basePath


In [15]:
def readArchive():
    with open(baseArchive(), 'r', encoding='utf-8') as archive:
        data = archive.readlines()
    return data

In [16]:
with open('E:/ESTUDOS/UFPel/TEC_VII/01_SEMANA/exemplo.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


In [17]:
url_base = 'http://localhost:8000/exemplo.html'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

model_url = requests.get(url_base, headers=headers)
print(model_url.text)


<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Portal de Videogames</title>
</head>
<body>
    <header>
        <h1>Bem-vindo ao Portal de Videogames</h1>
        <nav>
            <ul>
                <li><a href="#ultimas-noticias">Ãltimas NotÃ­cias</a></li>
                <li><a href="#analises">AnÃ¡lises</a></li>
                <li><a href="#videos">VÃ­deos</a></li>
            </ul>
        </nav>
    </header>

    <section id="ultimas-noticias">
        <h2>Ãltimas NotÃ­cias</h2>
        <article>
            <h3>Novo LanÃ§amento: "Cyberpunk 2077"</h3>
            <p>O altamente aguardado jogo "Cyberpunk 2077" jÃ¡ estÃ¡ disponÃ­vel para compra. Confira as anÃ¡lises!</p>
        </article>

        <article>
            <h3>AtualizaÃ§Ã£o de "Fortnite"</h3>
            <p>A mais recente atualizaÃ§Ã£o de "Fortnite" traz novos modos de jogo e skins exclusivas.</p>
       

In [18]:
with open('E:/ESTUDOS/UFPel/TEC_VII/01_SEMANA/exemplo.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')
print(soup.title.text)



Portal de Videogames


header - h1
section - h2
        - article - h3
                  - p

In [19]:
def get_info(model_url):
    response = requests.get(model_url)
    if response.status_code != 200:
        print(f"Erro ao acessar {model_url}: {response.status_code}")
        return None  # Retorna None se houver um erro na requisição

    soup = BeautifulSoup(response.content, 'html.parser')

    header = soup.find('h1').text.strip() if soup.find('h1') else 'N/A'
    print(f"Coletando dados de: {header}")  # Mostra o nome do modelo sendo coletado

    example_data = []

    sections = soup.find_all('section')
    for section in sections:
        seccion = section.find('h2').text.strip() if section.find('h2') else 'N/A'
        
        # Iterando sobre os artigos dentro de cada seção
        articles = section.find_all('article')
        for article in articles:
            subtitule = article.find('h3').text.strip() if article.find('h3') else 'N/A'
            describe = article.find('p').text.strip() if article.find('p') else 'N/A'

            example_data.append({
                'seção': seccion,
                'subtitulo': subtitule,
                'post': describe,
            })

    return example_data

In [20]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()  # Usa o arquivo padrão se nenhum nome for fornecido

    # Cria o DataFrame com os dados
    df = pd.DataFrame(data)

    try:
        # Verifica se o arquivo já existe
        if os.path.exists(filename):
            # Lê o cabeçalho do arquivo existente para verificar a compatibilidade
            existing_df = pd.read_csv(filename, nrows=0)
            if list(existing_df.columns) != list(df.columns):
                raise ValueError("As colunas do novo DataFrame não correspondem ao arquivo existente.")
        
        # Salva os dados no arquivo (adiciona ao final se já existir)
        df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)
        print(f"Dados salvos no arquivo '{filename}' com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar dados: {e}")


In [21]:
if __name__ == '__main__':
    # Certifique-se de que o servidor local está rodando com a URL fornecida
    url_base = 'http://localhost:8000/exemplo.html'
    data = get_info(url_base)

    if data:
        saveData(data)

Coletando dados de: Bem-vindo ao Portal de Videogames
Dados salvos no arquivo 'e:\CODES_01\PYTHON\NPL\PLN_TEC_VII\01_semana\db\dataExample.csv' com sucesso!
